In [1]:
from data import MNIST
import torch
from torch.utils.data import DataLoader

dataset = MNIST()
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, drop_last=True)


In [2]:
from model import Model
model = Model([1,28,28], 1000, 1e-4, 0.2)
xs = dataset[:5]
model.ELBO(xs)

torch.Size([999, 784])
torch.Size([999, 784])
torch.Size([999, 784])
torch.Size([999, 784])
torch.Size([999, 784])


2